<a href="https://colab.research.google.com/github/jasonasindi/git_test/blob/main/Project_1_(Code_1)_Induction_Motor_Failure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Project 1: Induction Motor Fault Detection

### **Data Pre-Processing**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns


cp = "/content/drive/MyDrive/imbalance"

# File path definitions

normal_file_names = glob.glob("/content/drive/MyDrive/normal"+'/normal/*.csv')
imnormal_file_names_6g = glob.glob(cp+'/imbalance/6g/*.csv')
imnormal_file_names_10g = glob.glob(cp+'/imbalance/10g/*.csv')
imnormal_file_names_15g = glob.glob(cp+'/imbalance\\15g/*.csv')
imnormal_file_names_20g = glob.glob(cp+'/imbalance\\20g/*.csv')
imnormal_file_names_25g = glob.glob(cp+'/imbalance\\25g/*.csv')
imnormal_file_names_30g = glob.glob(cp+'/imbalance\\30g/*.csv')

# Data Loading
def dataReader(path_names):
    data_n = pd.DataFrame()
    for i in path_names:
        low_data = pd.read_csv(i,header=None)
        data_n = pd.concat([data_n,low_data],ignore_index=True)
    return data_n

# Data loading for normal and imnormal data

data_n = dataReader(normal_file_names)
data_6g = dataReader(imnormal_file_names_6g)
data_10g = dataReader(imnormal_file_names_10g)
data_15g = dataReader(imnormal_file_names_15g)
data_20g = dataReader(imnormal_file_names_20g)
data_25g = dataReader(imnormal_file_names_25g)
data_30g = dataReader(imnormal_file_names_30g)

data_n.info()

# Down sampling

In [ ]:

# Downsampling the for reducing noise and computational load
def downSampler(data, a, b):
  data_decreased = pd.DataFrame()
  x = b
  for i in range(int(len(data)/x)):
    data_mean = data.iloc[a:b,:].sum() / x
    data_decreased = pd.concat([data_decreased, data_mean.to_frame().T])
    a += x
    b += x
  return data_decreased


In [ ]:
data_n = downSampler(data_n, 0, 2000)
data_6g = downSampler(data_6g, 0, 2000)
data_10g = downSampler(data_10g, 0, 2000)
data_15g = downSampler(data_15g, 0, 2000)
data_20g = downSampler(data_20g, 0, 2000)
data_25g = downSampler(data_25g, 0, 2000)
data_30g = downSampler(data_30g, 0, 2000)

In [ ]:
data_n # Displaying the downsampled 'data_n' DataFrame

FFT Convolve

In [ ]:
# Using Fast Fourier Transform Convolution because of the large data size
from scipy import signal
import pandas as pd
import numpy as np

# Function to compute the autocorrelation of data using FFT
def FFT(data):
    autocorr = signal.fftconvolve(data, data[::-1], mode='same')  # change mode to 'same'
    return pd.DataFrame(autocorr)

# List of datasets
datasets = [data_n, data_6g, data_10g, data_15g, data_20g, data_25g, data_30g]

# Apply FFT function to each dataset and concatenate them
data = pd.concat([FFT(dataset) for dataset in datasets], ignore_index=True)

# Create target labels for each dataset and concatenate them
y = pd.concat([pd.DataFrame(np.full((len(dataset), 1), i)) for i, dataset in enumerate(datasets)], ignore_index=True)

# Print data and labels
print(data)
print(y)

## Train the Model

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.20, shuffle=True)

In [ ]:
print("Shape of Train Data : {}".format(X_train.shape))
print("Shape of Test Data : {}".format(X_test.shape))

In [ ]:
# Sequential algorithm used because it perform better than other algorithms when trained on large datasets.
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],), kernel_initializer='random_uniform'),
    Dense(64, activation='relu', kernel_initializer='random_uniform'),
    Dense(128, activation='relu', kernel_initializer='random_uniform'),
    Dense(64, activation='relu', kernel_initializer='random_uniform'),
    Dense(32, activation='relu', kernel_initializer='random_uniform'),
    Dense(7, activation='softmax', kernel_initializer='random_uniform')])



# Compile

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Fit

In [ ]:
Ft = model.fit(X_train , y_train , epochs=40, validation_split=0.2)

## Data Visualization

In [ ]:
import matplotlib.pyplot as plt

# Plotting the accuracy of a model over epochs

plt.plot(Ft.history['accuracy'])
plt.plot(Ft.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()



## Prediction and Loss plot

In [ ]:

def plot_confusion_matrix(confusion_mtx, classes):
    # Create a heatmap from our confusion matrix
    sns.heatmap(confusion_mtx, annot=True, fmt='d', xticklabels=classes, yticklabels=classes)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
# Convert predictions classes to one hot vectors
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert validation observations to one hot vectors
y_true = np.argmax(y_test, axis=1)

# compute the confusion matrix
confusion_mtx = confusion_matrix(y_true, y_pred_classes)

# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(7))

# Plot training & validation loss values
plt.plot(Ft.history['loss'])
plt.plot(Ft.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()